In [9]:
import numpy as np
import cPickle
import nltk
import heapq
import os
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
import re
from nltk.stem.porter import *
stemmer = PorterStemmer()
print stemmer.stem('clean')

clean


# Visualize functions

In [10]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [11]:
def rule_q4(sen, ne):
    clean_ne = list(set(ne))
    remove_words = ['car', 'vehicl','improv','dealership','custom','receiv','satisfact','respond','servic','time',
                    'innova','center','facil','feel','ok','tell','problem','pay','dealer','attent','hurri','condit',
                    'ant','fine','deliver','get','question','deliveri','need','quality','day','side','kind','chang',
                    'honda','visit','told','speak','ask','requir','maruti','cleanli','henc','place','area','hand',
                    'compani','process','qualiti','care','outsid','complaint','depart','hour','wait','front','home',
                    'centr','system']
    clean_ne = [word for word in clean_ne if word not in remove_words and len(word)>1]    
    save_words = ['polish','wash','interior','extra','rupe','check','vacuum','clean','intern',
                  'insid','ac','dry','engin','inter']
    uni_words = ['mat','interior','charg']
    clean_ne = list(set(clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]))
    
    # rules to merge keywords:
    # rules to seperate keywords:
    if 'rupe' in clean_ne:
        clean_ne[clean_ne.index('rupe')] = 'charg'
    if 'rate' in clean_ne:
        clean_ne[clean_ne.index('rate')] = 'charg'        
    if 'cost' in clean_ne:
        clean_ne[clean_ne.index('cost')] = 'charg'
    if 'money' in clean_ne:
        clean_ne[clean_ne.index('money')] = 'charg'
    if 'insid' in clean_ne:
        clean_ne[clean_ne.index('insid')] = 'interior'
    if 'intern' in clean_ne:
        clean_ne[clean_ne.index('intern')] = 'interior'
    if 'window' in clean_ne:
        clean_ne[clean_ne.index('window')] = 'glass'
    if 'dirt' in clean_ne:
        clean_ne[clean_ne.index('dirt')] = 'dust'  
    if 'manag' in clean_ne:
        clean_ne[clean_ne.index('manag')] = 'staff'
    if 'advisor' in clean_ne:
        clean_ne[clean_ne.index('advisor')] = 'staff'  
    if 'supervisor' in clean_ne:
        clean_ne[clean_ne.index('supervisor')] = 'staff'  
    if 'labor' in clean_ne:
        clean_ne[clean_ne.index('labor')] = 'staff'
    if 'worker' in clean_ne:
        clean_ne[clean_ne.index('worker')] = 'staff'         
    if 'cloth' in clean_ne:
        clean_ne[clean_ne.index('cloth')] = 'dri'
    if 'vacuum' in clean_ne:
        clean_ne[clean_ne.index('vacuum')] = 'dri'        
    if 'water' in clean_ne:
        clean_ne[clean_ne.index('water')] = 'clean'
    if 'spot' in clean_ne:
        clean_ne[clean_ne.index('spot')] = 'stain'          
    if 'wash' in clean_ne:
        clean_ne[clean_ne.index('wash')] = 'clean'
    if len(clean_ne) > 1 and 'dust' in clean_ne:
        clean_ne.remove('dust')
    if len(clean_ne) > 1 and 'clean' in clean_ne:
        clean_ne.remove('clean')
        
    uniwrd = [cn for cn in clean_ne if cn in uni_words]
    if uniwrd != []:
        clean_ne = uniwrd
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new


def process_corpus(x1, pos_tags, general_stop,sent_ind): #new process_corpus func.===============================
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    r_ind = []
    for ind,x in enumerate(x1):
        if 'no improvement' in x:
            doc_noimprove.append(x)
            r_ind.append(ind)
        else:
            nn_list = []
            sen = x
            pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
            for token in pos_new:
                if token[1] in pos_tags and not token[0] in general_stop:
                    nn_list.append(token[0])
            nn_list = [stemmer.stem(word) for word in nn_list] #stemming
            nn_list = rule_q4(sen, nn_list) # apply rule
                    
            if nn_list != []:
                nn_extracted.append(nn_list)
                doc_nn.append(sen)
            else:
                doc_other.append(sen)
                r_ind.append(ind)
    sent_ind = [i for j, i in enumerate(sent_ind) if j not in r_ind]
    return doc_noimprove, [doc_nn, nn_extracted], doc_other, sent_ind


nn_corpus = []
# English stop words lists
# stop_words = stopwords.words('english')
stop_words = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself',
            'yourselves','he','him','his','himself','she','her','good','fr','rs','hers','herself',
            'it','its','itself','they','them','their','theirs','themselves','what','which','who',
            'whom','this','that','these','those','am','is','time','something','are','was','were',
            'be','been','do','does','did','doing','a','an','the','and','but','if','or','because',
            'as','until','while','of','at','by','for','take','better','ve','with','about','against',
            'into','through','during','before','to','from','in','out','on','off','over','under',
            'again','further','then','once', 'after','didn','don','ft','have','had','has','having','had',
            'here','there','when','where','why','how','all','any','both','each','few','more','most','other',
            'some','such','only','doesn','sq','own','same','so','than','too','very','can','will','just',
            'should','now','bit','anything','till','thing','things','toyota','hrs','km','sta','pm',
            'everything','feedback','part','parts','issue','issues','ask','way','use','give','giving',
            'gives','sometimes','focus','lot','work','works','need','needs','think','bring','people',
            'person']
punctuation_list = [unicode(i) for i in string.punctuation]

for punctuation in punctuation_list:
    stop_words.append(punctuation)

    
question_class = 'q4'
fname = '..//dataset//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of comments', len(content)
f.close()

length of comments 1146


In [12]:
# split comment with multi-sentence into multi-comments
content_new = []
sent_to_comm_id = []
for comment_ind, comment in enumerate(content):
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
    sent_to_comm_id += [comment_ind]*len(sents)

pos_tags = ['NN', 'NNS']
doc1, doc2, doc3, sent_comm_ind = process_corpus(content_new, pos_tags, stop_words, sent_to_comm_id)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'length of new content', len(content_new)
print 'signle sentence with nn', len(doc_nn)
print 'single sentence without nn', len(doc3)
print 'no comments', len(doc1)

length of new content 2725
signle sentence with nn 2213
single sentence without nn 492
no comments 20


# Document Frequency

In [21]:
from collections import Counter
def df_count(x1):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df

def filter_ne(test_corpus, df):  # assuming each review contain one aspect
    for xth, doc in enumerate(test_corpus):
        if len(doc)>1:
            df_words = [df[word] for word in doc]
            idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__)
            test_corpus[xth] = [stemmer.stem(doc[ith]) for ith in idx]
    return test_corpus

def write_file(corpus, idx_list, word):
    f1 = open('cluster/%s/%s_comment.txt' % (word, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()
        
def main_category(df_list, nn_clean, corpus,sc_ind): # new main_cate func. with common index
    if (not os.path.isdir("cluster")):
        os.mkdir("cluster")
    name_list = {}
    major_list = [word for word in df_list if df_list[word]>10]
    print "majot list:\n", major_list
    scidx_set = []
    for word in major_list:
        if not os.path.isdir("cluster/%s" %word):
            os.mkdir("cluster/%s" %word)
        idx_set = []
        for idx, doc in enumerate(nn_clean):
            if word in doc:
                idx_set.append(idx)
        write_file(doc_nn, idx_set, word)
        name_list[word] = idx_set
        scidx_set += [sc_ind[i] for i in idx_set]
    return name_list, len(set(scidx_set))

df = df_count(nn_extracted)
nn_clean = filter_ne(nn_extracted, df)
df = df_count(nn_clean)
# print df
dict_map = dict(df.most_common())
name_list,nb_comm = main_category(dict_map, nn_clean,doc_nn,sent_comm_ind)

print 'comment ratio', float(nb_comm)/len(content)*100

majot list:
[u'polish', 'staff', 'tire', 'stain', 'seat', 'dri', u'glass', 'check', u'engin', 'interior', 'mat', 'dust', u'charg', u'clean']
comment ratio 85.7766143106


# Split Large Cluster

In [14]:
from nltk.corpus import wordnet as wn
from itertools import product
def sim_check_word(word1,word2):
    syns1 = wn.synsets(word1)
    syns2 = wn.synsets(word2)
    sims = []
    for sense1, sense2 in product(syns1, syns2):
        d = wn.path_similarity(sense1, sense2)
        sims.append((d))
    return max(sims)

def sim_check_list(list1,list2):
    sims = []
    for word in list1:
        for word2 in list2:
            sims.append(sim_check_word(word, word2))
    return max(sims)


def l2_extract(corpus, idx_list):
    num = 0
    other_content = []
    adj_batchlist = []
    local_content = []
    for idx in idx_list:
        doc = corpus[idx]
        local_content.append(doc)
        adj_list = []
        for word in doc.split():
            try:
                tmp = [wn.synsets(word)[hh].pos() for hh in range(len(wn.synsets(word)))] 
            except IndexError:
                tmp = None
            if 'a' in tmp:
                adj_list.append(word)
        adj_batchlist.append(adj_list)
    
    
    return adj_batchlist, local_content

# tt_list, local_content = l2_extract(doc_nn, name_list['clean'])
# print tt_list